In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential,load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
output_folder = "CNN_Model_Output"
file = "Combined_Viral_Data.csv"
file2 = "HIV_HBV_ROTA_EBOLA_OVERALL.csv"

# Load the data
data = pd.read_csv(file)
data2 = pd.read_csv(file2)

# Combine data
combined_data = pd.concat([data, data2], ignore_index=True)

# Shuffle the combined data
shuffled_data = combined_data.sample(frac=1).reset_index(drop=True)

# Prepare data for training
X = np.array(shuffled_data['DNA'])
y = np.array(shuffled_data['Label'])

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            padded_sequence = sequence + 'N' * (max_length - len(sequence))
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    return padded_sequences

max_length = 11195

X_train_padded = pad_sequences(X_train, max_length)
X_test_padded = pad_sequences(X_test, max_length)


In [ ]:
# One-hot encoding for DNA sequences
def one_hot_encoding(seq):
    base_dict = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'C': [0, 0, 1, 0], 'G': [0, 0, 0, 1], 'N': [0, 0, 0, 0]}
    return np.array([base_dict.get(base, [0, 0, 0, 0]) for base in seq])

X_train_encoded = np.array([one_hot_encoding(seq) for seq in X_train_padded])
X_test_encoded = np.array([one_hot_encoding(seq) for seq in X_test_padded])
